<a href="https://colab.research.google.com/github/beyzoskaya/PyTorch_deepLearning/blob/main/GPS_IMU_LSTM_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
excel_file = 'imu_data.xlsx'
imu_df = pd.read_excel(excel_file)
imu_df.head()

In [ ]:
gyro_z = imu_df['Adis16497_gyro_z']
gyro_y = imu_df['Adis16497_gyro_y']
gyro_x = imu_df['Adis16497_gyro_x']
acceleration_z = imu_df['Adis16497_acc_z']
acceleration_y = imu_df['Adis16497_acc_y']
acceleration_x = imu_df['Adis16497_acc_x']
time_ms = imu_df['time(ms)']

In [ ]:
excel_file = 'gps_data.xlsx'
gps_df = pd.read_excel(excel_file)
gps_df.head()

In [ ]:
gps_boylam = gps_df['gps_boylam']
gps_enlem = gps_df['gps_enlem']
time_ms = gps_df['time(ms)']
print ('gps_boylam',gps_boylam.head())

In [ ]:
combined_df = pd.merge(imu_df, gps_df, on='time(ms)', how='inner')

In [ ]:
if combined_df.empty:
print("Eşleşen zaman bulunamadı. lstm çalışmayacak")
else:

  data = combined_df.sort_values(by='time(ms)')
  sequence_length = 70
  sequences =[]
  labels = []
  for i in range (len(data) - sequence_length +1):
    sequence_data = data.iloc [i:i + sequence_length ][ ['Adis16497_acc_x','Adis16497_acc_y',
    'Adis16497_acc_z','Adis16497_gyro_x', 'Adis16497_gyro_y','Adis16497_gyro_z']]

    label = data.iloc [i + sequence_length -1 ][['gps_enlem','gps_boylam']]
    sequences.append(sequence_data)
    labels.append(label)
  X = np.array(sequences)
  y = np.array(labels)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [ ]:
X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

In [ ]:
model = Sequential()
num_features = 9
#dynamic_seq_length = 2
#X_train_reshaped = X_train.reshape((X_train.shape[0], dynamic_seq_length, num_features))
model.add(LSTM(128, input_shape=(70, 6), activation='relu', return_sequences=True))
model.add(LSTM(units=128, activation='tanh', return_sequences=True, input_shape=(sequence_length,
num_features)))
model.add(LSTM(128, input_shape=(70, 6), activation='relu', return_sequences=True))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(LSTM(units=128, activation='tanh' ))
model.add(Dense(units=2)) # Çıkış boyutunu 2 olarak değiştirildi

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=20, validation_data=(X_test, y_test))